Below is the original Python code. Includes data cleaning, pre-processing and categorizing.

* AI assisted code writing. AI was also used to format the code with comments and better readability for portfolio purposes.
* Note that the code does not run 1:1 compared to what is seen in the Tableau dashboard. It is only for reference purposes. But, the logic is the same throughout the code.

In [ ]:
import pandas as pd
import numpy as np
import re

# ────────────────────────────────────────────────
# Configuration flag - controls whether we remove exact duplicate rows
# (same ID + Activity + Development Objective combination)
DEDUPLICATE_INPUT = True

# ────────────────────────────────────────────────
# Load the source Excel file
input_file = "input_file_example.xlsx" # In this case, the data source

try:
    # Read Excel, forcing certain columns to be read as strings (preserves leading zeros, etc.)
    df = pd.read_excel(
        input_file,
        sheet_name="Sheet1",
        converters={
            "ID": str,
            "Activity": str,
            "Development Objective": str
        }
    )
except Exception as e:
    print(f"Error reading Excel file: {e}")
    exit(1)

# ────────────────────────────────────────────────
# Define cleaning functions for each key column

def clean_activity(value):
    """Clean Activity column: remove control characters, handle various forms of missing/empty values"""
    value = str(value).strip()                         # Convert to string + remove leading/trailing whitespace
    value = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', value) # Remove all control characters
    # Treat various representations of "missing" as "N/A"
    if pd.isna(value) or value.lower() in ["", "nan", "none", "null", "\xa0"]:
        return "N/A"
    if value.lower() in ["n/a", "na"]:
        return "N/A"
    return value

def clean_id(value):
    """Clean ID column: similar logic but different default for missing values"""
    value = str(value).strip()
    value = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', value)
    if pd.isna(value) or value.lower() in ["", "nan", "none", "null", "\xa0"]:
        return "Unknown_ID"
    return value

def clean_dev_objective(value):
    """Clean Development Objective - almost same logic as Activity"""
    value = str(value).strip()
    value = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', value)
    if pd.isna(value) or value.lower() in ["", "nan", "none", "null", "\xa0"]:
        return "N/A"
    if value.lower() in ["n/a", "na"]:
        return "N/A"
    return value

# Apply cleaning functions to the three main columns
df["Activity"] = df["Activity"].apply(clean_activity)
df["ID"] = df["ID"].apply(clean_id)
df["Development Objective"] = df["Development Objective"].apply(clean_dev_objective)

# Extra safety: fill any remaining NaN values after cleaning
df["Activity"] = df["Activity"].fillna("N/A")
df["ID"] = df["ID"].fillna("Unknown_ID")
df["Development Objective"] = df["Development Objective"].fillna("N/A")

# ────────────────────────────────────────────────
# Show basic statistics before any deduplication
print(f"Initial input records: {len(df)}")
print(f"Unique IDs: {df['ID'].nunique()}")
print(f"Unique Activities: {df['Activity'].nunique()}")
print(f"Unique Development Objectives: {df['Development Objective'].nunique()}")

# ────────────────────────────────────────────────
# Check for exact duplicates on the business key (ID + Activity + Dev Objective)
id_activity_dev_counts = df.groupby(
    ["ID", "Activity", "Development Objective"]
).size().reset_index(name="count")

duplicates_id_activity_dev = id_activity_dev_counts[
    id_activity_dev_counts["count"] > 1
][["ID", "Activity", "Development Objective", "count"]]

if not duplicates_id_activity_dev.empty:
    print(f"Found {len(duplicates_id_activity_dev)} duplicate ID-Activity-Development Objective pairs:")
    print(duplicates_id_activity_dev.head(10))

# ────────────────────────────────────────────────
# Optional deduplication step (controlled by config flag)
if DEDUPLICATE_INPUT:
    initial_count = len(df)
    # Keep first occurrence only when all three key columns match
    df = df.drop_duplicates(subset=["ID", "Activity", "Development Objective"])
    print(f"Deduplicated input: {initial_count} → {len(df)} records")

# ────────────────────────────────────────────────
# Create a surrogate key that should be unique per logical record
# Format: ID_Activity_DevObjective_rowindex
df["Unique_Record_ID"] = df.apply(
    lambda x: f"{x['ID']}_{x['Activity']}_{x['Development Objective']}_{x.name}"
    if pd.notna(x['ID']) and pd.notna(x['Activity']) and pd.notna(x['Development Objective'])
    else f"Unknown_{x.name}", 
    axis=1
)

print(f"Unique Unique_Record_IDs after deduplication: {df['Unique_Record_ID'].nunique()}")

# ────────────────────────────────────────────────
# Save intermediate debug files
df.to_csv("input_debug.csv", index=False)
unique_activities = pd.DataFrame(df["Activity"].unique(), columns=["Activity"])
unique_activities.to_csv("unique_activities_debug.csv", index=False)

print("Raw input saved to 'input_debug.csv'")
print("Unique activities saved to 'unique_activities_debug.csv'")

# ────────────────────────────────────────────────
# Report how many records ended up with placeholder values
blank_activities = df[df["Activity"] == "N/A"][["ID", "Activity", "Development Objective", "Unique_Record_ID"]]
if not blank_activities.empty:
    print(f"Found {len(blank_activities)} N/A or blank activities:")
    print(blank_activities.head(10))

blank_ids = df[df["ID"] == "Unknown_ID"][["ID", "Activity", "Development Objective", "Unique_Record_ID"]]
if not blank_ids.empty:
    print(f"Found {len(blank_ids)} Unknown_ID assignments:")
    print(blank_ids.head(10))

blank_dev_objectives = df[df["Development Objective"] == "N/A"][["ID", "Activity", "Development Objective", "Unique_Record_ID"]]
if not blank_dev_objectives.empty:
    print(f"Found {len(blank_dev_objectives)} N/A Development Objectives:")
    print(blank_dev_objectives.head(10))

# ────────────────────────────────────────────────
# Keyword-based classification rules (multi-label possible)
activity_categories = {
    "Training / Workshop": ["REDACTED"],
    "Seminar": ["REDACTED"],
    "Self-Study": ["REDACTED"],
    "Language Exchange": ["REDACTED"],
    "Professional Development": ["REDACTED"],
    "Involvement in Projects": ["REDACTED"],
    "Mentoring / Coaching": ["REDACTED"],
    "Certification": ["REDACTED"],
    "Networking": ["REDACTED"],
    "Research": ["REDACTED"],
    "Safety / Compliance": ["REDACTED"],
    "Site Visit": ["REDACTED"],
    "Team Building": ["REDACTED"],
    "General Development": ["REDACTED"],
    "On-the-Job Learning": ["REDACTED"],
    "Operational Duties": ["REDACTED"],
    "Leadership / Management": ["REDACTED"],
    "Event Planning": ["REDACTED"]
}

def classify_activity(activity):
    """Rule-based classifier: returns list of matching categories (can be multiple)"""
    activity_clean = str(activity).lower().strip("- ")
    activity_clean = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', activity_clean)
    
    if activity_clean.lower() == "n/a":
        return ["Other"]
    
    matching_categories = set()
    for category, keywords in activity_categories.items():
        if any(keyword in activity_clean for keyword in keywords):
            matching_categories.add(category)
    
    return list(matching_categories) if matching_categories else ["Other"]

# ────────────────────────────────────────────────
# Create one row per category (exploding multi-category records)
new_rows = []
unique_inputs = set()

# Group by the business key + surrogate key
for (id_val, activity, dev_objective, unique_record_id), group in df.groupby(
    ["ID", "Activity", "Development Objective", "Unique_Record_ID"]
):
    unique_inputs.add((id_val, activity, dev_objective, unique_record_id))
    
    # Get list of categories for this activity
    categories = classify_activity(activity)
    if not categories:
        categories = ["Other"]
    
    # Take the first row of the group as base
    base_row = group.iloc[0].copy()
    
    # Create one new row per category
    for category in categories:
        new_row = base_row.copy()
        new_row["Activity Category"] = category
        new_rows.append(new_row)

# Convert list of dicts/series → final DataFrame
new_df = pd.DataFrame(new_rows)

# ────────────────────────────────────────────────
# Final statistics
print(f"Unique ID-Activity-Development Objective-Unique_Record_ID pairs in input: {len(unique_inputs)}")
print(f"Total output records (with multi-category duplicates): {len(new_df)}")
print(f"Unique Unique_Record_IDs in output: {new_df['Unique_Record_ID'].nunique()}")

other_activities = new_df[new_df["Activity Category"] == "Other"][
    ["ID", "Activity", "Development Objective", "Unique_Record_ID"]
]
print(f"Total activities classified as Other: {len(other_activities)}")
if not other_activities.empty:
    print("Sample of activities classified as Other:")
    print(other_activities.head(10))

# ────────────────────────────────────────────────
# Save results and debug files
new_df.to_excel("output_file_example.xlsx", index=False)
unique_pairs = pd.DataFrame(list(unique_inputs), columns=["ID", "Activity", "Development Objective", "Unique_Record_ID"])
unique_pairs.to_csv("unique_pairs_debug.csv", index=False)

category_counts = new_df.groupby("Activity Category")["Unique_Record_ID"].nunique().reset_index(name="Unique_Record_Count")
category_counts.to_csv("category_counts_debug.csv", index=False)

print("Output saved to 'output_file_example.xlsx'")
print("Unique pairs saved to 'unique_pairs_debug.csv'")
print("Category counts saved to 'category_counts_debug.csv'")

# ────────────────────────────────────────────────
# Show preview of final result
print("\nSample of categorized DataFrame:")
print(new_df[["ID", "Activity", "Development Objective", "Activity Category", "Unique_Record_ID"]].head(20))